In [46]:
import pandas as pd
import numpy as np
df = pd.read_csv("Train_B_Tree.csv")

In [47]:
class Node:
    def __repr__(self):
        return "<Node Object with attribute "+str(self.attr)+" and bound "+str(self.bound)+" and decision "+str(self.decision)+">"
    def __init__(self, *args, **kwargs):
        try:
            self.parent = kwargs['parent']
        except:
            self.parent = None
        try:
            self.left = kwargs['left']
        except:
            self.left = None
        try:
            self.right = kwargs['right']
        except:
            self.right = None
        try:
            self.decision = kwargs['decision']
        except:
            self.decision = None
        try:
            self.attr = kwargs['attr']
        except:
            self.attr = None
        try:
            self.bound = kwargs['bound']
        except:
            self.bound = None

In [48]:
def regressiontree(df):
    train=df.sample(frac=0.7) #random state is a seed value
    test=df.drop(train.index)
    list_attr = train.columns.to_numpy()[:-1].tolist()
    NodeList=[]
    root = Node()
    split(train, root, 'left', list_attr, NodeList)
    sqerr = 0
    for i in test.index:
        start = root.left
        print(type(start))
        try:
            while(not start.decision):
                if(test[start.attr][i]<=start.bound):
                    start = start.left
                if(test[start.attr][i]>start.bound):
                    start = start.right
        except:
            print(start)
        e_val = start.decision - test['csMPa'][i]
        print(e_val)
        sqerr = sqerr + e_val**2
    print(sqerr)


In [49]:
def LS_ERROR(train, list_attr):
    input_train = train[list_attr]
    if(len(input_train.drop_duplicates())<=1):
        return 0
    else:
        #t = type(train['csMPa'][0])
        m = train['csMPa'].mean()
        ERROR = 0
        #print(len(train))
        for i in train.index:
            t=type(i)
            ERROR = ERROR + (train['csMPa'][i] - m)**2
        return ERROR

In [50]:
def WEIGHTED_LS(left, right, list_attr):
    lls = LS_ERROR(left, list_attr)
    rls = LS_ERROR(right, list_attr)
    return (lls*len(left) + rls*len(right))

In [51]:
def split(root_train, parent, side, list_attr, NodeList):
    root_train = root_train.iloc[:, np.random.permutation(len(root_train.columns))]
    a = Node(parent=parent)
    if(side == 'left'):
        parent.left = a
    else:
        parent.right = a
    NodeList.append(a)
    input_train = root_train[list_attr]
    if(LS_ERROR(root_train, list_attr) == 0):
        a.decision = root_train['csMPa'].mean()
        return
    MIN_LS_ERROR = np.inf
    for attribute in list_attr:
        valid_set = root_train[attribute].unique().tolist()
        valid_set.sort()
        if(len(valid_set)>=2):
            for i in valid_set[:-1]:
                left_temp = root_train[root_train[attribute]<=i]
                right_temp = root_train[root_train[attribute]>i]
                LS = WEIGHTED_LS(left_temp, right_temp, list_attr)
                if(LS<MIN_LS_ERROR):
                    #print(LS[0])
                    left = left_temp
                    right = right_temp
                    attr = attribute
                    bound = i
                    MIN_LS_ERROR = LS
    print(MIN_LS_ERROR, attr, bound)
    a.attr = attr
    a.bound = bound
    split(left, a, 'left', list_attr, NodeList)
    split(right, a, 'right', list_attr, NodeList)
    return

In [52]:
#print(df[df['age']<=3])
if __name__ == "__main__":
    for i in range(1):
        regressiontree(df)
        
    

60359381.59759996 cement 312.7
15107990.0378 age 14
463643.59959999996 age 7
172009.70570000002 cement 236.0
38679.66510000001 cement 190.3
6143.568500000001 flyash 0.0
1045.3471 age 3
69.91549999999998 water 192.0
0.7991999999999999 slag 200.0
0.0036000000000000597 cement 116.0
0 cement 133.0
8.289800000000001 cement 108.3
0.00490000000000004 slag 212.0
0 cement 141.3
240.56070000000005 slag 212.0
56.702 cement 135.7
6.9458 water 192.0
0 cement 102.0
0 cement 108.3
2.0748 cement 183.9
0.2791999999999986 slag 116.0
0.0035999999999998464 cement 141.3
0 cement 170.3
0 cement 157.0
345.60080000000016 coarseaggregate 1055.6
67.72980000000003 slag 93.4
0.6984000000000022 slag 0.0
0 cement 165.0
0 cement 167.0
1.7160999999999966 slag 129.8
0 cement 165.0
2.029900000000001 slag 0.0
0 cement 166.1
0.006400000000000012 cement 168.0
0 cement 168.9
8885.659700000002 fineaggregate 799.5
3551.6748 water 185.7
1012.798 cement 213.8
187.20929999999996 coarseaggregate 1053.5
0.039999999999999716 cemen